# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
import platform
if (platform.system() == "Darwin"):
    env = UnityEnvironment(file_name="Tennis.app")
elif (platform.system() == "Linux"):
    env = UnityEnvironment(file_name="Tennis_Linux_NoVis/Tennis.x86_64")
else:
    print("specify path to Unity environment based on your OS")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


In [5]:
from IPython.display import clear_output, display
from ipywidgets import IntSlider, Output
from matplotlib import pyplot as plt
import collections
%matplotlib inline

def live_plot(out, data_dict, figsize=(9,6), title=''):
    with out:
        clear_output(wait=True)
        plt.figure(figsize=figsize)
        for label,data in data_dict.items():
            plt.plot(data, label=label)
        plt.title(title)
        plt.grid(True)
        plt.ylabel('Score')
        plt.xlabel('Episode #')
        plt.legend(loc='upper left') # the plot evolves to the right
        plt.show()

In [6]:
import torch
from collections import deque

def ddpg(agent, n_episodes=1000):
    
    out_plot = Output()
    out_text = Output()
    display(out_plot)
    display(out_text)
    
    avr_scores_deque = deque(maxlen=100)
    all_scores = []
    avr_scores = []
    plot_data = collections.defaultdict(list)

    for i_episode in range(1, n_episodes+1):
        episode_scores, steps = agent.steps()
  
        episode_score = np.max(episode_scores)
        all_scores.append(episode_score)
        avr_scores_deque.append(episode_score)
        avr_score = np.mean(avr_scores_deque)
        avr_scores.append(avr_score)
        
        if i_episode == 250:
            agent.add_noise_on_act(False)
        
        plot_data['episode score'].append(episode_score)
        plot_data['average score'].append(avr_score)
        live_plot(out_plot, plot_data)
        
        with out_text: print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, avr_score, episode_score), end="")               
                    
        if i_episode % 100 == 0:
            with out_text: print('\rEpisode {:d}\tAverage Score: {:.2f}'.format(i_episode, avr_score))
        if avr_score > 0.5:
            with out_text: print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, avr_score))
            break
                                   
    return all_scores, avr_scores

In [7]:
import time
from ddpg_agent import DdpgAgent
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

config = dict(env=dict(state_size=state_size,
                      action_size=action_size,
                      simulator=env,
                      brain_name=brain_name,
                      num_agents=num_agents),
             actor=dict(hidden_layers=[256, 128, 64]),
             critic=dict(hidden_layers=[256, 128, 64]),
             learning=dict(buffer_size=int(50000),
                          batch_size=1024,
                          discount=0.9,
                          max_t=1000,
                          soft_update_tau=0.2,
                          lr_actor=0.001,
                          lr_critic=0.001,
                          lr_decay=dict(activate=False,
                                       actor_step=100,
                                       actor_gamma=0.5,
                                       critic_step=100,
                                       critic_gamma=0.5),
                          learn_every_n_steps=1,
                          num_learn_steps=1,
                          checkpointfile="checkpoint.pth"
                          )
             )

agent = DdpgAgent(config, 2, device)

start_time = time.time()
scores, avr_scores = ddpg(agent, n_episodes=500)
elapsed = time.time() - start_time
print("Agent training time: {:.1f} min" .format(elapsed/60))

agent.save_model()

Output()

Output()

Agent training time: 10.6 min


In [ ]:
from model import Actor

checkpoint = torch.load("checkpoint.pth")
trained_model = Actor(checkpoint["state_size"],
                      checkpoint["action_size"],
                      2,
                      checkpoint["hidden_layers"])
trained_model.load_state_dict(checkpoint["state_dict"])

def play(n_episodes=1, max_steps=1000):
    scores = []
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=False)[brain_name] # reset the environment    
        states = np.array(env_info.vector_observations)    # get the current state
        states = torch.from_numpy(states).float().to("cpu")
        scores = np.zeros(num_agents)       # initialize the score (for each agent)
        trained_model.eval()                # set train= False
        while True:
            with torch.no_grad():
                actions = trained_model(states).cpu().data.numpy()
            env_info = env.step(actions)[brain_name]
            next_states = np.array(env_info.vector_observations)
            rewards = env_info.rewards
            dones = env_info.local_done
            scores += env_info.rewards
            states = torch.from_numpy(next_states).float().to("cpu")
            if np.any(dones):
                break
        print("Episode: {}, Score: {:.2f}" .format(i_episode, np.mean(scores)))

play(10)

When finished, you can close the environment.

In [ ]:
env.close()